In [19]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config(path="Users/odl_user_244707/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 42, Finished, Available)

Workspace name: quick-starts-ws-244707
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-244707


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "projectrubric"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws,name=cluster_name)
    print("found existing cluster use it")
except ComputeTargetException:
    print("creating a new compute")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",min_nodes=1,max_nodes=4)
    compute_target = ComputeTarget.create(ws,cluster_name,compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())



StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 43, Finished, Available)

found existing cluster use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-12-03T15:12:07.219000+00:00', 'errors': None, 'creationTime': '2023-12-03T15:10:47.191349+00:00', 'modifiedTime': '2023-12-03T15:10:56.924652+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
%pip install azureml-sdk 

StatementMeta(, , -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 63.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.0 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.6/656.6 KB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 KB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 KB 26.9 MB/s eta 0:00:00
     

In [26]:
import os 
cur_d = os.getcwd()
print(cur_d)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 49, Finished, Available)

/synfs/notebook/0/aml_notebook_mount


In [35]:
#from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C':choice(0.001,0.01,1,10,20,50,100,200,500,1000),
        '--max_iter':choice(50,100,200,300)
    }
)

# Specify a Policy
 ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = './training'  
script_file = os.path.join(script_folder,'train.py')


# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_244707/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
 ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='./Users/odl_user_244707',script='train.py',compute_target=compute_target,environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,policy=policy,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,max_concurrent_runs=4) 

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 58, Finished, Available)

In [24]:
import os  
cur_dir = os.getcwd()
print(cur_dir)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 47, Finished, Available)

/synfs/notebook/0/aml_notebook_mount


In [8]:
%pip install azureml-widgets

StatementMeta(, , -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/nfs4/pyenv-575d8c99-64c0-4a38-be47-ad157d163db2/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [37]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
run_hyp = exp.submit(hyperdrive_config)
RunDetails(run_hyp).show()
run_hyp.wait_for_completion(show_output=True)
assert(run_hyp.get_status()=="Completed")

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 60, Finished, Available)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22
Web View: https://ml.azure.com/runs/HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-244707/workspaces/quick-starts-ws-244707&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-12-03T17:14:32.438159][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-12-03T17:14:32.9260660Z][SCHEDULER][INFO]Scheduling job, id='HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_0' 
[2023-12-03T17:14:33.0011304Z][SCHEDULER][INFO]Scheduling job, id='HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_1' 
[2023-12-03T17:14:33.1292647Z][SCHEDULER][INFO]Scheduling job, id='HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_2' 
[2023-12-03T17:14:33.127799][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-12-03T17:14:33.2260649Z][SCHEDULER][INFO]Scheduling job, id='HD_eca738bc-f660-4c7b-920f-36b6ab6b9b

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 61, Finished, Available)

In [31]:
import joblib

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 54, Finished, Available)

In [38]:
import joblib
# Get your best run and save the model from that run.
print(run_hyp.get_children_sorted_by_primary_metric(top=0,reverse=False,discard_no_metric=False))
best_run = run_hyp.get_best_run_by_primary_metric()
#get_metrics()
#Returns the metric from all the runs that were launched by this HyperdriveRun.
print("Best run metrics " ,best_run.get_metrics())
### YOUR CODE HERE ###
#get_details()
#Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
#get_file_names()
#Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 62, Finished, Available)

[{'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_13', 'hyperparameters': '{"--C": 10, "--max_iter": 300}', 'best_primary_metric': 0.9176024279210926, 'status': 'Completed'}, {'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_1', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.9174506828528073, 'status': 'Completed'}, {'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_19', 'hyperparameters': '{"--C": 10, "--max_iter": 100}', 'best_primary_metric': 0.9169954476479515, 'status': 'Completed'}, {'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_12', 'hyperparameters': '{"--C": 20, "--max_iter": 300}', 'best_primary_metric': 0.9169954476479515, 'status': 'Completed'}, {'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_5', 'hyperparameters': '{"--C": 200, "--max_iter": 200}', 'best_primary_metric': 0.9169954476479515, 'status': 'Completed'}, {'run_id': 'HD_eca738bc-f660-4c7b-920f-36b6ab6b9b22_6', 'hyperparameters': '{"--C": 50, "--max_iter": 300}', 'best_p

In [39]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 63, Finished, Available)

In [41]:
import os 
cu_d = os.getcwd()
print(cu_d)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 65, Finished, Available)

/synfs/notebook/0/aml_notebook_mount


In [43]:
chang_dir = 'Users/odl_user_244707/'
os.chdir(chang_dir)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 67, Finished, Available)

In [44]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 68, Finished, Available)

In [49]:
import os 
cu_d = os.getcwd()
print(cu_d)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 73, Finished, Available)

/synfs/notebook/0/aml_notebook_mount/Users/odl_user_244707


In [50]:
%cd ..

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 74, Finished, Available)

/synfs/notebook/0/aml_notebook_mount/Users


In [51]:
%cd ..

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 75, Finished, Available)

/synfs/notebook/0/aml_notebook_mount


In [60]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 84, Finished, Available)

In [61]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config,show_output=False)
#from azureml.widgets import RunDetails
#RunDetails(remote_run).show()
remote_run.wait_for_completion()

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 85, Finished, Available)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5520758b-160e-4815-b592-806fab770c97,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_5520758b-160e-4815-b592-806fab770c97',
 'target': 'projectrubric',
 'status': 'Completed',
 'startTimeUtc': '2023-12-03T17:46:15.798235Z',
 'endTimeUtc': '2023-12-03T18:23:04.308856Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'projectrubric',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-244707","workspace_name":"quick-starts-ws-244707","region":"southcentralus","compute_target":"projec

In [62]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#get_metrics
#Returns the metrics
print("Best run metrics :",best_run.get_metrics())
#get_details()
#Returns a dictionary with details for the run
print("Best run details : ",best_run.get_details())

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 86, Finished, Available)

The model you attempted to retrieve requires 'azureml-train-automl-runtime' to be installed at '==1.53.2'. Please install 'azureml-train-automl-runtime==1.53.2' (e.g. `pip install azureml-train-automl-runtime==1.53.2`) and then rerun the previous command.
Run(Experiment: udacity-project,
Id: AutoML_5520758b-160e-4815-b592-806fab770c97_37,
Type: azureml.scriptrun,
Status: Completed)
None
Best run metrics : {'norm_macro_recall': 0.4371276755154334, 'AUC_micro': 0.9807799521507965, 'precision_score_weighted': 0.9059900421779148, 'accuracy': 0.9148710166919576, 'log_loss': 0.18477291615729186, 'recall_score_micro': 0.9148710166919576, 'average_precision_score_macro': 0.8254781523696038, 'f1_score_micro': 0.9148710166919576, 'f1_score_macro': 0.751684758824199, 'average_precision_score_weighted': 0.9557548350268407, 'average_precision_score_micro': 0.9816152057287895, 'f1_score_weighted': 0.9078713376714261, 'recall_score_macro': 0.7185638377577167, 'precision_score_macro': 0.80518126442637

In [63]:
best_run.get_metrics()

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 87, Finished, Available)

{'norm_macro_recall': 0.4371276755154334,
 'AUC_micro': 0.9807799521507965,
 'precision_score_weighted': 0.9059900421779148,
 'accuracy': 0.9148710166919576,
 'log_loss': 0.18477291615729186,
 'recall_score_micro': 0.9148710166919576,
 'average_precision_score_macro': 0.8254781523696038,
 'f1_score_micro': 0.9148710166919576,
 'f1_score_macro': 0.751684758824199,
 'average_precision_score_weighted': 0.9557548350268407,
 'average_precision_score_micro': 0.9816152057287895,
 'f1_score_weighted': 0.9078713376714261,
 'recall_score_macro': 0.7185638377577167,
 'precision_score_macro': 0.8051812644263784,
 'balanced_accuracy': 0.7185638377577167,
 'AUC_macro': 0.9486186555466819,
 'precision_score_micro': 0.9148710166919576,
 'weighted_accuracy': 0.9636331183888713,
 'recall_score_weighted': 0.9148710166919576,
 'AUC_weighted': 0.948618655546682,
 'matthews_correlation': 0.5162279583197743,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_5520758b-160e-4815-b592-806fab770c97_3

In [64]:
fitted_model

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 88, Finished, Available)

In [65]:
best_run

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 89, Finished, Available)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5520758b-160e-4815-b592-806fab770c97_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [67]:
compute_target.delete()

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 91, Finished, Available)

KeyError: 'azure-asyncoperation'